# Builds a LangChain chatbot

In [45]:
import enum
from langchain_core.messages.base import BaseMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import logging as log
import os
import pprint

In [46]:
log.basicConfig(level=log.INFO, format='%(asctime)s [%(levelname)5s] %(message)s',datefmt='%H:%M:%S')

In [47]:
import os

class LlmVendor( enum.Enum):
    Google=1,
    OpenAI=2

class LlmModel( enum.Enum):
    GeminiFlash=1,
    GeminiPro=2
    OpenAI=3
    
class LlmModelFactory():
    def __init__(self, model: LlmModel, vendor: LlmVendor, envVar:str):
        self._model = model
        self._vendor= vendor
        self._envVar= envVar

        if not envVar in os.environ:
            raise ValueError(f'{envVar} not found in the environment.')

    @staticmethod
    def createGoogleGeminiFlash(): 
        return LlmModelFactory(model= LlmModel.GeminiFlash,  
                               vendor= LlmVendor.Google,
                               envVar='GOOGLE_API_KEY')
    @staticmethod
    def modelName( llmModel: LlmModel):
        match llmModel:
            case LlmModel.GeminiFlash:
                return "gemini-1.5-flash"
            case LlmModel.GeminiPro:     
                return "gemini-1.5-pro"
        
    def __str__(self):
        return f'(model:{self._model}, vendor:{self._vendor}, envVar:{self._envVar})'

    def createChatModel(self, 
                       maxOutputTokens=250,
                       temperature=0.0):
        match self._vendor:
            case LlmVendor.Google:
                return ChatGoogleGenerativeAI(model= LlmModelFactory.modelName(self._model), 
                                              max_output_tokens=maxOutputTokens, 
                                              temperature=temperature)
                

llmModelFactory= LlmModelFactory.createGoogleGeminiFlash()
print(llmModelFactory)

(model:LlmModel.GeminiFlash, vendor:LlmVendor.Google, envVar:GOOGLE_API_KEY)


In [48]:
model = llmModelFactory.createChatModel()

In [49]:
messages= [('system','You are a helpful assistant. That can translate from English to {language}'),
           ('human','Translate the following text:\n{text}')]

In [50]:
chain = ChatPromptTemplate(messages) | model | StrOutputParser()

In [51]:
result= chain.invoke({"language": "Italian", "text": "hi!"})

In [52]:
result

'Ciao!'

## Other test

In [61]:
chainModelToParser = model | StrOutputParser()

In [62]:
messages= [('system','You are a helpful assistant. That can translate from English to {language}'),
           ('human','Translate the following text:\n{text}')]
promptTemplate= ChatPromptTemplate(messages)
chain= promptTemplate | chainModelToParser

In [63]:
result= chain.invoke({"language": "Italian", "text": "How are you?"})
result

"How are you?  translates to:\n\n**Come stai?** (informal, used with friends and family)\n\nor\n\n**Come sta?** (formal, used with strangers or people you don't know well)"

# Chatbot. It runs until the user types 'quit'

In [66]:
messages= [('system','You are a polite and helpful assistant.')]
chainModelToParser = model | StrOutputParser()

while True:
    for (role,message) in messages:
        print(f'Role:   {role}')
        print(f'Message:{message}\n')
    print('Enter the next prompt. quit to terminate:')
    userPrompt= input()
    if 'quit'==userPrompt.strip().lower():
        break
    messages.append( ('human',userPrompt))  
    promptTemplate= ChatPromptTemplate(messages)
    chain= promptTemplate | chainModelToParser
    result= chain.invoke({})
    print(result)
    messages.append( ('ai',result))  

Role:   system
Message:You are a polite and helpful assistant.

Enter the next prompt:


 Who is the current President of the USA?


The current President of the USA is Joe Biden.
Role:   system
Message:You are a polite and helpful assistant.

Role:   human
Message:Who is the current President of the USA?

Role:   ai
Message:The current President of the USA is Joe Biden.

Enter the next prompt:


 How is his wife called?


President Biden's wife's name is Jill Biden.
Role:   system
Message:You are a polite and helpful assistant.

Role:   human
Message:Who is the current President of the USA?

Role:   ai
Message:The current President of the USA is Joe Biden.

Role:   human
Message:How is his wife called?

Role:   ai
Message:President Biden's wife's name is Jill Biden.

Enter the next prompt:


 quit
